# ID-3决策树模型

## 1.模型代码

### 1.1 决策树各节点定义

In [12]:
# -*- coding: utf-8 -*
## 定义节点
class Node(object): 
    def __init__(self, attr_init=None, label_init=None, attr_down_init={} ):  
        self.attr = attr_init  
        self.label = label_init 
        self.attr_down = attr_down_init 

### 1.2 决策树实现流程

In [23]:
#构建树的函数（训练函数）
import pandas as pd
data_file_encode = "gb18030"
with open("watermelon_3.csv", mode = 'r', encoding = data_file_encode) as data_file:
    data = pd.read_csv(data_file)
def TreeGenerate(df):
    ''' 
    Branching for decision tree using recursion 
     
    @param df: the pandas dataframe of the data_set
    @return root: Node, the root node of decision tree
    '''  
    # generating a new root node
    new_node = Node(None, None, {})
    label_arr = df[df.columns[-1]]
    
    label_count = NodeLabel(label_arr)
    if label_count:  # assert the label_count isn's empty
        new_node.label= max(label_count, key=label_count.get) 
            
        # end if there is only 1 class in current node data
        # end if attribution array is empty
        if len(label_count) == 1 or len(label_arr) == 0:
            return new_node
        
        # get the optimal attribution for a new branching
        new_node.attr, div_value = OptAttr(df)  # via Gini index 
        
        # recursion
        if div_value == 0:  # categoric variable
            value_count = ValueCount(data[new_node.attr]) 
            for value in value_count:
                df_v = df[ df[new_node.attr].isin([value]) ]  # get sub set
                # delete current attribution
                df_v = df_v.drop(new_node.attr, 1)  
                if len(df_v) == 0:
                    new_node.attr_down[value] = Node(None,new_node.label, {})
                    return new_node
                else:
                    new_node.attr_down[value] = TreeGenerate(df_v)     #迭代创立下一级节点
                
        else:  # continuous variable # left and right child
            value_l = "<=%.3f" % div_value
            value_r = ">%.3f" % div_value
            df_v_l = df[ df[new_node.attr] <= div_value ]  # get sub set
            df_v_r = df[ df[new_node.attr] > div_value ]
 
            new_node.attr_down[value_l] = TreeGenerate(df_v_l)
            new_node.attr_down[value_r] = TreeGenerate(df_v_r)
        
    return new_node

### 1.3 决策树的预测

In [24]:
'''
根据根进行预测

@param root：节点，决策树的根节点 
@param df_sample：数据框，样本行
'''
def Predict(root, df_sample):
    try:
        import re   # 使用正则表达式获取字符串中的数字
    except ImportError :
        print("module re not found")
    
    while root.attr != None :        
        # 连续变量
        if df_sample[root.attr].dtype == float :
            # 从 root.attr_down 获取 div_value
            for key in list(root.attr_down):
                num = re.findall(r"\d+\.?\d*",key)
                div_value = float(num[0])
                break
            if df_sample[root.attr].values[0] <= div_value:
                key = "<=%.3f" % div_value
                root = root.attr_down[key]
            else:
                key = ">%.3f" % div_value
                root = root.attr_down[key]
                
        # 分类变量
        else:  
            key = df_sample[root.attr].values[0]
            # 检查子分支中的属性是否
            if key in root.attr_down: 
                root = root.attr_down[key]
            else: 
                break
            
    return root.label 

### 1.4 对目标和属性的取值进行保存和计数

In [25]:
'''
计算目标类别及其计数

@param label_arr：目标类别的数据数组 
@return label_count: dict，出现的目标各类和它的计数
'''  
def NodeLabel(label_arr):
    label_count = {}       # 存储目标各类别数量
      
    for label in label_arr:
        if label in label_count: label_count[label] += 1
        else: label_count[label] = 1
        
    return label_count

'''
计算分类属性的出现值及其计数 

@param data_arr：属性的数据数组
@return value_count: dict，出现的值和它的计数
'''
def ValueCount(data_arr):
    value_count = {}       # store count of value 
      
    for label in data_arr:
        if label in value_count: value_count[label] += 1
        else: value_count[label] = 1
        
    return value_count

### 1.5 根据信息增益选择当前数据集下最优属性

In [26]:
'''
找到当前数据集的最优属性

@param df：data_set 的 pandas 数据框 
@return opt_attr：分支的最佳属性 
@return div_value：对于离散变量值为0 对于连续变量值为t值为二分法的划分值
'''  
def OptAttr(df):
    info_gain = 0
    
    for attr_id in df.columns[1:-1]:
        info_gian_tmp, div_value_tmp = InfoGain(df, attr_id) # 调用信息增益计算函数 
        if  info_gian_tmp > info_gain :  # 找到增益最大的那个属性和划分值
            info_gain = info_gian_tmp
            opt_attr = attr_id
            div_value = div_value_tmp
        
    return opt_attr, div_value

### 1.6 计算信息增益（连续数据采用二分法实现）

In [27]:
'''
ID3算法采用信息增益最大化来实现最优划分属性的选择，这里主要的挑战是离散和连续两种属性变量的分别操作。

对于离散变量（categoric variable），参考书p75-77内容实现，
对于连续变量（continuous variable），采用书p83-85所介绍的二分法实现。

相关内容如信息熵、信息增益最大化、二分法等可参考书p75-76及p84页内容。

'''  
def InfoGain(df, index):
    info_gain = InfoEnt(df.values[:,-1])  # 所有类别的信息熵
    div_value = 0  # 连续属性的分割值（二分法）
    
    n = len(df[index])  # 数据集的长度
    # 1.对连续变量使用二分法
    if df[index].dtype == float:
        sub_info_ent = {}  # 存储分割值及其子集熵
        
        df = df.sort_values([index], ascending=1)  # 对数值进行排序
        df = df.reset_index(drop=True)
        
        data_arr = df[index]
        label_arr = df[df.columns[-1]]
        
        for i in range(n-1):
            div = (data_arr[i] + data_arr[i+1]) / 2
            sub_info_ent[div] = ( (i+1) * InfoEnt(label_arr[0:i+1]) / n ) \
                              + ( (n-i-1) * InfoEnt(label_arr[i+1:-1]) / n )
        # 我们的目标是获得最小子集熵和它的划分值在p84公式
        div_value, sub_info_ent_max = min(sub_info_ent.items(), key=lambda x: x[1])
        info_gain -= sub_info_ent_max
        
    # 2.对于离散变量（分类变量）直接计算信息增益
    else:
        data_arr = df[index]
        label_arr = df[df.columns[-1]]
        value_count = ValueCount(data_arr)
            
        for key in value_count:
            key_label_arr = label_arr[data_arr == key]
            info_gain -= value_count[key] * InfoEnt(key_label_arr) / n
    
    return info_gain, div_value

### 1.7 计算信息熵

In [28]:
'''
计算属性的信息熵 
'''  
def InfoEnt(label_arr):
    try :
        from math import log2
    except ImportError :
        print("module math.log2 not found")
    
    ent = 0
    n = len(label_arr)
    label_count = NodeLabel(label_arr) # label_count {'是': 8, '否': 9}
    
    for key in label_count:
        ent -= ( label_count[key] / n ) * log2( label_count[key] / n )
    
    return ent

### 1.8 将树构建为图，然后进行可视化

In [29]:
def TreeToGraph(i, g, root):
    '''
    从根开始构建图
    
    @param i：这棵树中的节点号 
    @param g: pydotplus.graphviz.Dot() 对象 
    @param root：根节点 @return i：修改后的节点号
    @return g: 修改后的 pydotplus.graphviz.Dot() 对象 
    @return g_node：graphviz 中的当前根节点
    '''
    try:
        from pydotplus import graphviz
    except ImportError:
        print("module pydotplus.graphviz not found")

    if root.attr == None:
        g_node_label = "Node:%d\n好瓜:%s" % (i, root.label)
    else:
        g_node_label = "Node:%d\n好瓜:%s\n属性:%s" % (i, root.label, root.attr)
    g_node = i
    g.add_node(graphviz.Node(g_node, label=g_node_label, fontname="FangSong"))

    for value in list(root.attr_down):
        i, g_child = TreeToGraph(i + 1, g, root.attr_down[value])
        g.add_edge(graphviz.Edge(g_node, g_child, label=value, fontname="FangSong"))

    return i, g_node

def DrawPNG(root, out_file):
    '''
    从根开始可视化决策树
    
    @变量root：节点，树的根节点。 
    @变量out_file: str，输出文件的名称和路径
    '''
    try:
        from pydotplus import graphviz
    except ImportError:
        print("module pydotplus.graphviz not found")
        
    g = graphviz.Dot()  # generation of new dot   

    TreeToGraph(0, g, root)
    g2 = graphviz.graph_from_dot_data( g.to_string() )
    
    g2.write_png(out_file) 

## 2. ID-3决策树模型使用

### 2.1 导入数据，并使用决策树模型

In [30]:
#导入包和数据
import matplotlib.pyplot as plt
import pandas as pd
data_file_encode = "gb18030"  
with open("watermelon_3.csv", mode = 'r', encoding = data_file_encode) as data_file:
    df = pd.read_csv(data_file)

root = TreeGenerate(df)

### 2.2 k折交叉验证 计算模型的泛化能力

In [31]:
accuracy_scores = []


# k折交叉验证

n = len(df.index)
k = 10
for i in range(k):                        
    m = int(n/k)
    test = []
    for j in range(i*m, i*m+m):
        test.append(j)                                                        # 随机抽取m个测试集
           
    df_train = df.drop(test)                                                  #划分训练集和测试集
    df_test = df.iloc[test]                 
    root = TreeGenerate(df_train)                               #训练并生成决策树
    
    # 计算决策树在测试集上的精度
    pred_true = 0
    for i in df_test.index:
        label = Predict(root, df[df.index == i])
        if label == df_test[df_test.columns[-1]][i]:
            pred_true += 1       
    accuracy = pred_true / len(df_test.index)
    accuracy_scores.append(accuracy) 
 

### 2.3 计算平均精度

In [32]:
# 计算平均精度
accuracy_sum = 0
print("accuracy: ", end = "")
for i in range(k):
    print("%.3f  " % accuracy_scores[i], end = "")
    accuracy_sum += accuracy_scores[i]
print("\naverage accuracy: %.3f" % (accuracy_sum/k))


root = TreeGenerate(df)                                 # 用 pydotplus.graphviz函数实现可视化  
DrawPNG(root, "decision_tree_ID3.png")

accuracy: 1.000  1.000  1.000  1.000  1.000  0.000  0.000  1.000  0.000  1.000  
average accuracy: 0.700


### 2.4 ID-3决策树可视化
![](decision_tree_ID3.png)